In [24]:
from TopoAnalysis import dem as d
from river_inversion import run_river_inversion

# Load your data
area = d.Area.load('/Users/Glong1/Desktop/Andes/Andes_watersheds/RapelRiver/rapel_area_utm30m')
print('area loaded')
fd = d.FlowDirectionD8.load('/Users/Glong1/Desktop/Andes/Andes_watersheds/RapelRiver/rapel_fd_utm30m')
print('fd loaded')
elevation = d.Elevation.load('/Users/Glong1/Desktop/Andes/Andes_watersheds/RapelRiver/rapel_SRTMGL130m_dem_utm.tif')
print('elevation loaded')

# Run analysis on a tributary (provide outlet UTM coords)
results = run_river_inversion(
    area=area,
    fd=fd,
    elevation=elevation,
    outlet_coords=(250212.9354, 6237948.848),  # Replace with actual coordinates
    m=0.45,
    A0=1e7,  # 10 km² (Goren's default)
    Gamma=10,
    q=10,
    block_uplift=True,
    bootstrap=True,
    bootstrap_iterations=100,
    plot_results=True
)

# With calibration data
calibration = {
    'H': 1000,  # Total uplift in meters
    't_H': 5e6   # Age in years
}

results_cal = run_river_inversion(
    area=area, fd=fd, elevation=elevation,
    outlet_coords=(x, y),
    calibration_data=calibration,
    #save_results='my_results.npz'
)

area loaded
fd loaded
elevation loaded
RIVER PROFILE INVERSION ANALYSIS

[1/7] Extracting tributary network...


TypeError: 'zip' object is not subscriptable